In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import math

import sys
sys.path.append('../')
import tree_lib.util as util

In [2]:
def func_power(f, x, exp):
    ret = x
    for _ in range(exp):
        ret = f(ret)
    return ret

def steps_to_target_iter(f, x, target):
    steps = 0
    curr_val = x
    while curr_val != target:
        curr_val = f(curr_val)
        steps += 1
    return steps

def largest_value_fitting(f, n, use_binsearch=True):
    assert n > 0
    if use_binsearch:
        if n == 1:
            return 1
        a = 0
        b = n
        while True:
            r = (a+b)//2
            rsq = f(r)
            if rsq <= n and f(r+1) > n:
                return f(r)
            else:
                if rsq <= n:
                    a = r
                else:
                    b = r
    else:
        curr_largest = 1
        while f(curr_largest) <= n:
            curr_largest += 1
        return f(curr_largest-1)

def steps_to_target_remainder(f, x, target, use_largest_fitting=True, use_binsearch=True):
    steps = 0
    curr_val = x
    while curr_val > target:
        curr_val -= largest_value_fitting(f, curr_val, use_binsearch) if use_largest_fitting else f(curr_val)
        steps += 1
    return steps

In [3]:
NUMBER_OF_SAMPLES = 4000
MAX_VALUE = 2**100
SKIP = MAX_VALUE // NUMBER_OF_SAMPLES

columns = ["number", "sqrt", "cubert", "div2", "div4", "div8", "log2", "log10", "gauss"]
data_vertical = np.array([
    [
        x,
        steps_to_target_iter(lambda x: math.floor(math.sqrt(x)), x, 1),
        steps_to_target_iter(lambda x: math.floor(x**(1./3.)), x, 1),
        steps_to_target_iter(lambda x: x // 2, x, 0),
        steps_to_target_iter(lambda x: x // 4, x, 0),
        steps_to_target_iter(lambda x: x // 8, x, 0),
        steps_to_target_iter(lambda x: math.floor(math.log2(x)), x, 0),
        steps_to_target_iter(lambda x: math.floor(math.log10(x)), x, 0),
        steps_to_target_iter(lambda x: math.floor(math.log10(x)), x, 0),
    ]
for x in range(1, MAX_VALUE, SKIP)])
data_horizontal = np.array([
    [
        x,
        steps_to_target_remainder(lambda x: x**2, x, 0),
        steps_to_target_remainder(lambda x: x**3, x, 0),
        steps_to_target_remainder(lambda x: x // 2, x, 1, use_largest_fitting=False),
        steps_to_target_remainder(lambda x: x // 4, x, 3, use_largest_fitting=False),
        steps_to_target_remainder(lambda x: x // 8, x, 7, use_largest_fitting=False),
        steps_to_target_remainder(lambda x: 2**x, x, 0, use_binsearch=False),
        steps_to_target_remainder(lambda x: 10**x, x, 0, use_binsearch=False),
        steps_to_target_remainder(lambda x: (x*(x+1))//2, x, 0, use_binsearch=True)
    ]
for x in range(1, MAX_VALUE, SKIP)])

In [4]:
df_vertical = pd.DataFrame(data=data_vertical, columns=columns)
fig = px.line(df_vertical, x="number", y=columns[1:], labels={"index": "Number", "value": "Steps to zero"}, title="Vertical convergence rate")
fig.show() 

In [5]:
df_horizontal = pd.DataFrame(data=data_horizontal, columns=columns)
fig = px.line(df_horizontal, x="number", y=columns[1:], labels={"index": "Number", "value": "Steps to zero"}, title="Horizontal convergence rate")
fig.show() 